In [1]:
import json
from collections import Counter
from typing import Any
from collections import defaultdict


file_path = 'naturalproofs_proofwiki.json'

with open(file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

In [2]:

def print_structure(obj: Any, indent: int = 0, max_depth: int = 3):
    """
    Recursively print the structure of a JSON object up to a certain depth.
    """
    prefix = '  ' * indent
    if isinstance(obj, dict):
        for k, v in obj.items():
            print(f"{prefix}{k}: {type(v).__name__}")
            if indent < max_depth:
                print_structure(v, indent + 1, max_depth)
    elif isinstance(obj, list):
        print(f"{prefix}List[{len(obj)}] of {type(obj[0]).__name__}" if obj else f"{prefix}Empty List")
        if indent < max_depth and obj:
            print_structure(obj[0], indent + 1, max_depth)
    else:
        print(f"{prefix}{repr(obj)} ({type(obj).__name__})")

def summarize_theorems(theorems):
    print(f"Total theorems: {len(theorems)}\n")
    
    key_counter = Counter()
    proof_lengths = []

    for thm in theorems:
        key_counter.update(thm.keys())
        if 'proofs' in thm:
            proof_lengths.append(len(thm['proofs']))
    
    print("Top-level keys in theorem objects:")
    for key, count in key_counter.items():
        print(f"  - {key}: appears in {count} theorems")

    print(f"\nAverage number of proofs per theorem: {sum(proof_lengths)/len(proof_lengths):.2f}")
    print(f"Max proofs in a theorem: {max(proof_lengths)}")

    print("\nSample structure of one theorem:")
    print_structure(theorems[0], max_depth=3)

    print("\nSample structure of one proof:")
    if 'proofs' in theorems[0] and theorems[0]['proofs']:
        print_structure(theorems[0]['proofs'][0], max_depth=3)


if "dataset" in data and "theorems" in data["dataset"]:
    theorems = data["dataset"]["theorems"]
    summarize_theorems(theorems)
else:
    print("Unexpected structure. Top-level keys are:")
    print_structure(data)


Total theorems: 19734

Top-level keys in theorem objects:
  - id: appears in 19734 theorems
  - type: appears in 19734 theorems
  - label: appears in 19734 theorems
  - title: appears in 19734 theorems
  - categories: appears in 19734 theorems
  - contents: appears in 19734 theorems
  - refs: appears in 19734 theorems
  - ref_ids: appears in 19734 theorems
  - proofs: appears in 19734 theorems
  - toplevel_categories: appears in 19734 theorems
  - recursive_categories: appears in 19734 theorems

Average number of proofs per theorem: 0.97
Max proofs in a theorem: 32

Sample structure of one theorem:
id: int
  0 (int)
type: str
  'theorem' (str)
label: str
  'Closed Form for Triangular Numbers' (str)
title: str
  'Closed Form for Triangular Numbers' (str)
categories: list
  List[4] of str
    'Triangular Numbers' (str)
contents: list
  List[2] of str
    'The [[Definition:Closed-Form Expression|closed-form expression]] for the $n$th [[Definition:Triangular Number|triangular number]] is:'

In [3]:
def explore_json_structure(data):

    # Print top-level structure
    print("\nTop-level JSON structure:")
    for key in data.keys():
        print(f"- {key} ({type(data[key]).__name__})")

    # Look inside dataset
    if "dataset" in data:
        dataset = data["dataset"]
        print("\n'dataset' keys:")
        for key in dataset:
            print(f"- {key} ({type(dataset[key]).__name__})")

        if "theorems" in dataset:
            all_entries = dataset["theorems"]
            print(f"\nNumber of entries in 'theorems': {len(all_entries)}")

            type_counter = Counter(entry.get("type", "MISSING") for entry in all_entries)
            print("\nUnique 'type' values and counts:")
            for t, count in type_counter.most_common():
                print(f"  - {t}: {count}")
        else:
            print("'theorems' key not found inside dataset.")
    else:
        print("'dataset' key not found in JSON.")

explore_json_structure(data)  # Replace with your filename


Top-level JSON structure:
- dataset (dict)
- splits (dict)

'dataset' keys:
- theorems (list)
- definitions (list)
- others (list)
- retrieval_examples (list)

Number of entries in 'theorems': 19734

Unique 'type' values and counts:
  - theorem: 19734


In [4]:
def collect_keys_types(obj: Any, key_types: defaultdict):
    if isinstance(obj, dict):
        for k, v in obj.items():
            key_types[k].add(type(v).__name__)
            collect_keys_types(v, key_types)
    elif isinstance(obj, list):
        for item in obj:
            collect_keys_types(item, key_types)

def explore_all_keys():

    key_types = defaultdict(set)
    collect_keys_types(data, key_types)

    print("\nUnique keys and value types found across the entire JSON structure:\n")
    for key in sorted(key_types.keys()):
        types_seen = ", ".join(sorted(key_types[key]))
        print(f"- {key}: {types_seen}")

explore_all_keys()


Unique keys and value types found across the entire JSON structure:

- categories: list
- contents: list
- dataset: dict
- definitions: list
- examples: list
- id: int
- label: str
- others: list
- proofs: list
- recursive_categories: list
- ref_ids: list
- refs: list
- retrieval_examples: list
- splits: dict
- test: dict
- theorems: list
- title: str
- toplevel_categories: list
- train: dict
- type: str
- valid: dict


In [5]:
def collect_type_values(obj: Any, type_counter: Counter):
    if isinstance(obj, dict):
        if 'type' in obj and isinstance(obj['type'], str):
            type_counter[obj['type']] += 1
        for v in obj.values():
            collect_type_values(v, type_counter)
    elif isinstance(obj, list):
        for item in obj:
            collect_type_values(item, type_counter)

def find_unique_types(data):

    type_counter = Counter()
    collect_type_values(data, type_counter)

    print("Unique values for 'type' and their counts:\n")
    for t, count in type_counter.most_common():
        print(f"- {t}: {count}")

find_unique_types(data)

Unique values for 'type' and their counts:

- theorem: 19734
- definition: 12420
- other: 1006


In [6]:
def strip_keys(obj, keys_to_remove):
    if isinstance(obj, dict):
        new_obj = {}
        for k, v in obj.items():
            if k not in keys_to_remove:
                new_obj[k] = strip_keys(v, keys_to_remove)
        return new_obj
    elif isinstance(obj, list):
        return [strip_keys(item, keys_to_remove) for item in obj]
    else:
        return obj

# Keys to remove
keys_to_strip = {"splits", "examples", "retrieval_examples"}

# Apply the cleaner
cleaned_data = strip_keys(data, keys_to_strip)

In [ ]:
# Extract and merge "theorems" and "definitions"
theorems = data.get("dataset", {}).get("theorems", [])
definitions = data.get("dataset", {}).get("definitions", [])

keys_to_remove = {"label", "toplevel_categories"}

# Function to clean a list of entries
def clean_entries(entries, keys_to_remove):
    return [
        {k: v for k, v in entry.items() if k not in keys_to_remove}
        for entry in entries
    ]

def normalize_contents_field(entries):
    for entry in entries:
        if isinstance(entry.get("contents"), list):
            entry["contents"] = "\n".join(entry["contents"])
        
        if "proofs" in entry:
            for proof in entry["proofs"]:
                if isinstance(proof.get("contents"), list):
                    proof["contents"] = "\n".join(proof["contents"])
    return entries


cleaned_theorems = normalize_contents_field(clean_entries(theorems, keys_to_remove))
cleaned_definitions = normalize_contents_field(clean_entries(definitions, keys_to_remove))

# Combine them into one flat list
combined_entries = cleaned_theorems + cleaned_definitions

# Optional sanity check
print(f"Merged entries: {len(combined_entries)} total — {len(theorems)} theorems, {len(definitions)} definitions")

Merged entries: 32154 total — 19734 theorems, 12420 definitions


In [8]:
with open('naturalproofs_cleaned.json', 'w', encoding='utf-8') as f:
    json.dump(combined_entries, f, indent=2, ensure_ascii=False)